## 1. OpenAIEmbeddings 클래스를 사용해 문장 임베딩을 해보세요.

— 선택 —
4. 자신이 생각하는 가장 깔끔한 트랜스포머 모델을 파이토치로 구현하고 리얼한 데이터셋으로 학습과 추론을 해보세요.

5. 4번의 코드를 바탕으로 트랜스포머의 얼개를 설명하는 보고서를 작성해보세요.

In [ ]:
import os
api_key = "Open_API_KEY"
os.environ['OPENAI_API_KEY'] = api_key

from openai import OpenAI

client = OpenAI()

In [ ]:
response = client.embeddings.create(
    input= "안녕하세요. 제 이름은 Kai 입니다.",
    model = "text-embedding-3-small"
)

embedded = response.data[0].embedding
print(len(embedded))

1536


In [ ]:
corpus = ["저는 한국 사람 입니다.",
          "오늘 날씨가 너무 좋아요.",
          "내일 일정은 어떻게 되는지 아시나요?"] # 여러 문장을 한번에 임베딩

response = client.embeddings.create(
    input = corpus,
    model = "text-embedding-3-small"
)

embedded_result = [data.embedding for data in response.data]
print("출력 임베딩 개수:",len(embedded_result))
print("임베딩 차원:",len(embedded_result[0]))

출력 임베딩 개수: 3
임베딩 차원: 1536


## 2. 사용자의 상황이나 배경을 제공해주면 그에 따라 이메일 양식을 만들어주는 챗봇을 만들어보세요.
ex) 거래처에 새해 인사와 함께 신규 계약건에 대해 이메일을 보내야한다.

In [ ]:
from transformers import AutoTokenizer
import transformers
import torch

HF_TOKEN = "HuggingFace_Token"

# 모델로딩
model = "meta-llama/Meta-Llama-3-8B-Instruct" #meta-llama/Meta-Llama-3-8B-Instruct 모델
tokenizer = AutoTokenizer.from_pretrained(
    model,
    token=HF_TOKEN,    #토큰 입력부
)

pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
    token=HF_TOKEN,    #토큰 입력부
)

messages = [
    {"role": "system", "content": "You need to send an email to a client with New Year's greetings about a new contract, but you need to address the email to a public audience. Answer in Korean unless otherwise specified"},
    {"role": "user", "content": "My name is Kai, please write Alex a mail with New Year's greetings and let him know that the newly developed large-scale LLM model is finished learning and available for delivery."},
]

terminators = [
    pipeline.tokenizer.eos_token_id,
    pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = pipeline(
    messages,
    max_new_tokens=512,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)
print(outputs[0]["generated_text"][-1])

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

...

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


{'role': 'assistant', 'content': "Here is an email to Alex with New Year's greetings:\n\nSubject: Happy New Year! - Update on Large-Scale LLM Model\n\nDear Alex,\n\nHappy New Year! As we step into a new year, I would like to take this opportunity to express my gratitude for the trust and support you have shown us throughout the year. We are thrilled to start this new chapter together.\n\nI am excited to inform you that our team has successfully completed the development of the large-scale LLM model, which has finished learning and is now available for delivery. This achievement is a testament to our team's dedication and expertise, and we are confident that it will bring significant value to your business.\n\nWe believe that this model will be a game-changer for your company, and we are eager to see the innovative applications you will come up with. Our team is committed to providing you with the best possible support and services to ensure a smooth integration and successful implement

In [ ]:
generated_text = outputs[0]["generated_text"][-1]["content"]

import textwrap

def pretty_print(text, width=80):
    wrapped_text = textwrap.fill(text, width=width)
    print(wrapped_text)

pretty_print(generated_text)

Here is an email to Alex with New Year's greetings:  Subject: Happy New Year! -
Update on Large-Scale LLM Model  Dear Alex,  Happy New Year! As we step into a
new year, I would like to take this opportunity to express my gratitude for the
trust and support you have shown us throughout the year. We are thrilled to
start this new chapter together.  I am excited to inform you that our team has
successfully completed the development of the large-scale LLM model, which has
finished learning and is now available for delivery. This achievement is a
testament to our team's dedication and expertise, and we are confident that it
will bring significant value to your business.  We believe that this model will
be a game-changer for your company, and we are eager to see the innovative
applications you will come up with. Our team is committed to providing you with
the best possible support and services to ensure a smooth integration and
successful implementation.  Once again, thank you for your conti

## 3. 최신 뉴스를 브리핑 해주는 챗봇을 만들어보세요.

In [ ]:
import pandas as pd

# 빅카인즈 사이트를 통해 3/17~3/18 기간 동안 경제 카테고리 뉴스 다운로드
# https://www.bigkinds.or.kr

df_news = pd.read_excel('NewsResult_20250317-20250318.xlsx')
print(df_news.columns)
df_news = df_news[['일자', '언론사', '기고자','제목', '본문']]
df_news.head()

/usr/local/lib/python3.10/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Index(['뉴스 식별자', '일자', '언론사', '기고자', '제목', '통합 분류1', '통합 분류2', '통합 분류3',
       '사건/사고 분류1', '사건/사고 분류2', '사건/사고 분류3', '인물', '위치', '기관', '키워드',
       '특성추출(가중치순 상위 50개)', '본문', 'URL', '분석제외 여부'],
      dtype='object')


,일자,언론사,기고자,제목,본문
0,20250318,데일리안,데일리안 이소영 기자 (sy@dailian.co.kr),아스트라제네카는 왜 알테오젠에 2조원을 베팅했을까,알테오젠이 글로벌 빅파마인 아스트라제네카(AZ)와 2조원 규모의 초대형 계약을 체결...
1,20250318,아시아투데이,NaN,"[포토]포멜라토, ‘디 아트 오브 누도’ 팝업 개최...우아한 매력의 배우 임수정",아시아투데이 박상선 기자 = 이탈리아 주얼리 브랜드 메종 포멜라토(Pomellato...
2,20250318,이데일리,이소현,"5분 충전에 400㎞ 달린다 진격의 BYD, 주가 사상 최고치",[이데일리 이소현 기자] 중국 전기차 제조업체 BYD(비야디)가 5분 만에 400㎞...
3,20250318,내일신문,김상범,미 러 화해 기대에 러 채권 루블 투자 ‘꿈틀’,"서방 투자자들, NDF로 제재 우회 \n러 기업채권 기대감도 상승 중 \n\n도널드..."
4,20250318,뉴스핌,송은정,"한국P&G, 상반기 채용전환형 인턴 모집 내달 13일 서류 마감",[서울=뉴스핌] 송은정 기자 = 한국P&G가 오는 4월 13일까지 2025년 상반기...


In [ ]:
from langchain import hub
from langchain.chat_models import init_chat_model
from langchain_openai import OpenAIEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_core.documents import Document
from langchain_community.document_loaders import DataFrameLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict

# LLM 정의
llm = init_chat_model("gpt-4o-mini", model_provider="openai")
# Embedding 모델 정의
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
# Vector store
vector_store = InMemoryVectorStore(embeddings)

In [ ]:
# Dataframe data loader
loader = DataFrameLoader(df_news, page_content_column = "본문")
data = loader.load()
data[:5]

[Document(metadata={'일자': 20250318, '언론사': '데일리안', '기고자': '데일리안 이소영 기자 (sy@dailian.co.kr)', '제목': '아스트라제네카는 왜 알테오젠에 2조원을 베팅했을까'}, page_content="알테오젠이 글로벌 빅파마인 아스트라제네카(AZ)와 2조원 규모의 초대형 계약을 체결했다. 이로써 알테오젠은 글로벌 빅파마와의 총 계약 규모를 9조원으로 늘리는 것은 물론, '특허 분쟁 리스크' 또한 원만히 해결될 가능성을 높였다.\n\n\n지난 17일 알테오젠은 아스트라제네카의 자회사인 메드이뮨 미국 영국 법인과 피하주사(SC) 제형 변경 플랫폼 ‘ALT-B.."),
 Document(metadata={'일자': 20250318, '언론사': '아시아투데이', '기고자': nan, '제목': '[포토]포멜라토, ‘디 아트 오브 누도’ 팝업 개최...우아한 매력의 배우 임수정'}, page_content='아시아투데이 박상선 기자 = 이탈리아 주얼리 브랜드 메종 포멜라토(Pomellato)가 누도(Nudo) 컬렉션 신제품 론칭을 기념해 18일 서울 서초구 신세계백화점 강남점에서 ‘디 아트 오브 누도(The Art of Nudo)’ 팝업 행사를 개최했다.\n\n 이날 행사에는 배우 임수정과 걸그룹 있지(ITZY)의 멤버 예지가 참석해 포토타임을 갖고 포멜라토의..'),
 Document(metadata={'일자': 20250318, '언론사': '이데일리', '기고자': '이소현', '제목': '5분 충전에 400㎞ 달린다 진격의 BYD, 주가 사상 최고치'}, page_content='[이데일리 이소현 기자] 중국 전기차 제조업체 BYD(비야디)가 5분 만에 400㎞ 주행이 가능한 초고속 배터리 충전 기술을 공개하며 전기차 시장에서 게임체인저로 떠올랐다.\n\n‘슈퍼 e-플랫폼’을 소개하고 있는 왕촨푸 BYD 회장(사진=BYD 웨이보 갈무리)\n\n블룸버그통신에 따르면 홍콩 증시에 상장된 BYD 주가는 18일

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(data)

_ = vector_store.add_documents(documents=all_splits)

# Define prompt for question-answering
prompt = hub.pull("rlm/rag-prompt")


# Define state for application
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str

/usr/local/lib/python3.10/dist-packages/langsmith/client.py:253: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [ ]:
# Define application steps
def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}

# Compile application and test
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

In [ ]:
response = graph.invoke({"question": "최근에 중국 전기차와 관련된 뉴스 요약해줘"})
print(response["answer"])

최근 BYD가 자사의 '슈퍼 e-플랫폼'을 통해 새로운 전기차 모델 두 가지를 출시하고 다음달 판매를 시작한다고 발표했습니다. 이 플랫폼은 충전 시간을 대폭 단축할 수 있는 배터리와 충전 시설을 포함하여, 전기차 대중화의 장벽을 낮추고 있습니다. 그러나 중국 시장에서 현지 전기차 제조업체들의 성장이 가파르며, 글로벌 완성차 업체들은 경쟁력을 확보하기 위해 전략을 재조정하고 있습니다.


In [ ]:
data[2] #실제 기사 내용(중국 전기차 제조 업체 BYD 관련 기사)

Document(metadata={'일자': 20250318, '언론사': '이데일리', '기고자': '이소현', '제목': '5분 충전에 400㎞ 달린다 진격의 BYD, 주가 사상 최고치'}, page_content='[이데일리 이소현 기자] 중국 전기차 제조업체 BYD(비야디)가 5분 만에 400㎞ 주행이 가능한 초고속 배터리 충전 기술을 공개하며 전기차 시장에서 게임체인저로 떠올랐다.\n\n‘슈퍼 e-플랫폼’을 소개하고 있는 왕촨푸 BYD 회장(사진=BYD 웨이보 갈무리)\n\n블룸버그통신에 따르면 홍콩 증시에 상장된 BYD 주가는 18일 개장과 동시에 최대 6% 상승해..')